# View All Page Contents

In [5]:
import requests
from bs4 import BeautifulSoup

url = 'https://gis.summitcountyco.gov/map/DetailData.aspx?Schno=6507888'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="https://www.w3.org/1999/xhtml">
<head>
<!-- Global Site Tag (gtag.js) Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-150931768-1"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());
        
        gtag('config', 'UA-150931768-1')
    </script>
<link href="styles/MainBlue2.css" rel="Stylesheet" type="text/css"/><link href="SCfavicon.ico" rel="SHORTCUT ICON"/><title>
	Detail Data
</title></head>
<body alink="aqua" bgcolor="#f7f7f7" class="table.DetailData" link="blue" vlink="blue"><br/>
</body>
</html>
<table class="DetailData">
<tr>
<td align="center" class="style1aSched" colspan="9">Schedule # 6507888</td>
</tr><tr>
<td class="style1" rowspan="2">Property Desc:</td><td class="style2" rowspan="2">BLDG 51 PR

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_data(url):
    # Send request and parse HTML
    r = requests.get(url)
    print(f'Status code = {r.status_code}\n')
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find tables
    tables = {
        'DetailData': soup.find('table', class_='DetailData'),
        'ValueData': soup.find('table', class_='ValueData'),
        'ImpData': soup.find('table', class_='ImpData'),
        'LandData': soup.find('table', class_='LandData')
    }

    # Function to extract data from a table cell
    def extract_cell_data(table, label):
        if not table:
            print(f"{label} table not found")
            return None
        cell = table.find('td', string=lambda text: text and label in text.strip())
        if cell:
            next_cell = cell.find_next('td')
            if next_cell:
                return next_cell.text.strip()
            print(f"{label} value cell not found")
        else:
            return None

    # Function to process a table and extract multiple fields
    def process_table(table, table_name, labels):
        if table:
            print(f"Found {table_name}\n")
            results = {}
            for label, display_name in labels:
                value = extract_cell_data(table, label)
                if value is not None:  # Allow empty strings
                    results[display_name] = value
                    print(f"{display_name}: {value}")
            return results
        else:
            print(f"{table_name} not found")
            return {}

    # Define labels to extract from DetailDataTable
    detail_labels = [
        ('Property Desc:', 'Property Description'),
        ('Phys. Address:', 'Physical Address'),
        ('Primary:', 'Primary Ownership'),
        ('Secondary:', 'Secondary Ownership'),
        ('C/O', 'Mailing Address - C/O'),
        ('Addr.', 'Address'),
        ('CSZ', 'Address - CSZ'),
    ]

    # Process DetailDataTable
    process_table(tables['DetailData'], 'DetailDataTable', detail_labels)

extract_data('https://gis.summitcountyco.gov/map/DetailData.aspx?Schno=6507888')

Status code = 200

Found DetailDataTable

Property Description: BLDG 51 PROSPECTOR VILLAGE COMMON AREA & CLUBHOUSE FKA LOT 33 WILDERNEST FIL 2
Physical Address: 0050 WILDERNEST CT (CR 1271) BLDG 51
Primary Ownership: PROSPECTOR VILLAGE HOMEOWNERS ASSOC,
Secondary Ownership: 
Mailing Address - C/O: 
Address - CSZ: FRISCO CO 804435265


# Find Most Recent Sale Date  

I don't feel like figuring out how to do this at the moment, so we're gonna use some AI slop for the time being

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

def extract_sale_dates(html_content):
    """
    Extract sale dates from property detail HTML using BeautifulSoup.
    
    Args:
        html_content (str): HTML content as string
    
    Returns:
        list: List of dictionaries containing sale information
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the DetailData table
    detail_table = soup.find('table', class_='DetailData')
    if not detail_table:
        return []
    
    sale_records = []
    
    # Find all cells with class 'style2b' which contain the sale dates
    date_cells = detail_table.find_all('td', class_='style2b')
    
    for cell in date_cells:
        cell_text = cell.get_text(strip=True)
        # Look for date pattern (M/D/YYYY or MM/DD/YYYY)
        if '/' in cell_text and len(cell_text.split('/')) == 3:
            try:
                # Validate it's actually a date
                date_obj = datetime.strptime(cell_text, '%m/%d/%Y')
                
                # Find the parent row to get related data
                parent_row = cell.find_parent('tr')
                if parent_row:
                    row_cells = parent_row.find_all('td')
                    
                    # Extract reception number, document type, and sale price
                    reception = ''
                    doc_type = ''
                    sale_price = ''
                    
                    for i, row_cell in enumerate(row_cells):
                        if row_cell == cell:  # Found our date cell
                            # Reception is typically before the date
                            if i > 0:
                                reception = row_cells[i-1].get_text(strip=True)
                            # Doc type is typically after the date
                            if i+1 < len(row_cells):
                                doc_type = row_cells[i+1].get_text(strip=True)
                            # Sale price is typically after doc type
                            if i+2 < len(row_cells):
                                sale_price = row_cells[i+2].get_text(strip=True)
                            break
                    
                    sale_records.append({
                        'reception_number': reception,
                        'sale_date': cell_text,
                        'date_object': date_obj,
                        'document_type': doc_type,
                        'sale_price': sale_price
                    })
                    
                    # Return immediately after finding the first date
                    break
                    
            except ValueError:
                continue
    
    return sale_records

def print_sale_dates(sale_records):
    """Print formatted sale date information."""
    if not sale_records:
        print("No sale dates found.")
        return
    
    print(f"Found {len(sale_records)} sale records:")
    print("-" * 60)
    
    for i, record in enumerate(sale_records, 1):
        print(f"{i}. Sale Date: {record['sale_date']}")
        print(f"   Reception: {record['reception_number']}")
        print(f"   Doc Type: {record['document_type']}")
        print(f"   Price: ${record['sale_price']}")
        print()

# Example usage
if __name__ == "__main__":
    # Read HTML content from file or string
    with open('html_output.txt', 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Extract sale dates
    sale_dates = extract_sale_dates(html_content)
    
    # Print results
    print_sale_dates(sale_dates)
    
    # Just the dates as a simple list
    dates_only = [record['sale_date'] for record in sale_dates]
    print("Sale dates only:", dates_only)

Found 1 sale records:
------------------------------------------------------------
1. Sale Date: 8/26/2016
   Reception: 1120146
   Doc Type: EAS
   Price: $0

Sale dates only: ['8/26/2016']


# Integrate Sales Date Into `extract_data` Function

Again, I don't feel like figuring it out right now, so, again... AI slop

In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def find_tables(url):
    # Send request and parse HTML
    r = requests.get(url)
    print(f'Status code = {r.status_code}\n')
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find tables
    tables = {
        'DetailData': soup.find('table', class_='DetailData'),
        'ValueData': soup.find('table', class_='ValueData'),
        'ImpData': soup.find('table', class_='ImpData'),
        'LandData': soup.find('table', class_='LandData')
    }

    # Function to extract sale date from DetailData table
    def extract_first_sale_date(table):
        if not table:
            return None
            
        # Find all cells with class 'style2b' which contain the sale dates
        date_cells = table.find_all('td', class_='style2b')
        
        for cell in date_cells:
            cell_text = cell.get_text(strip=True)
            # Look for date pattern (M/D/YYYY or MM/DD/YYYY)
            if '/' in cell_text and len(cell_text.split('/')) == 3:
                try:
                    # Validate it's actually a date
                    date_obj = datetime.strptime(cell_text, '%m/%d/%Y')
                    return cell_text  # Return the first valid date found
                except ValueError:
                    continue
        return None

    # Function to extract data from a table cell
    def extract_cell_data(table, label):
        if not table:
            print(f"{label} table not found")
            return None
        cell = table.find('td', string=lambda text: text and label in text.strip())
        if cell:
            next_cell = cell.find_next('td')
            if next_cell:
                return next_cell.text.strip()
            print(f"{label} value cell not found")
        else:
            print(f"{label} cell not found")
        return None

    # Function to process a table and extract multiple fields
    def process_table(table, table_name, labels):
        if table:
            print(f"Found {table_name}\n")
            results = {}
            for label, display_name in labels:
                if label == 'Sale Date':
                    # Use special function for sale date extraction
                    value = extract_first_sale_date(table)
                else:
                    value = extract_cell_data(table, label)
                
                if value is not None:  # Allow empty strings
                    results[display_name] = value
                    print(f"{display_name}: {value}")
            return results
        else:
            print(f"{table_name} not found")
            return {}

    # Define labels to extract from DetailDataTable
    detail_labels = [
        ('Sale Date', 'Most Recent Sale Date'),
        ('Property Desc:', 'Property Description'),
        ('Phys. Address:', 'Physical Address'),
        ('Primary:', 'Primary Ownership'),
        ('Secondary:', 'Secondary Ownership'),
        ('C/O', 'Mailing Address - C/O'),
        ('Addr:', 'Address'),
        ('CSZ', 'Address - CSZ')
    ]

    # Process DetailDataTable
    return process_table(tables['DetailData'], 'DetailDataTable', detail_labels)

# Example usage
if __name__ == "__main__":
    find_tables('https://gis.summitcountyco.gov/map/DetailData.aspx?Schno=6507888')

Status code = 200

Found DetailDataTable

Most Recent Sale Date: 8/26/2016
Property Description: BLDG 51 PROSPECTOR VILLAGE COMMON AREA & CLUBHOUSE FKA LOT 33 WILDERNEST FIL 2
Physical Address: 0050 WILDERNEST CT (CR 1271) BLDG 51
Primary Ownership: PROSPECTOR VILLAGE HOMEOWNERS ASSOC,
Secondary Ownership: 
Mailing Address - C/O: 
Address: PO BOX 5265
Address - CSZ: FRISCO CO 804435265
